<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace,OpenAIChatCompletionsModel, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

In [6]:
google_api_key = os.getenv('gemini_api_key')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

from agents import set_tracing_export_api_key
set_tracing_export_api_key(os.environ["open_api_key"])

### Let's start by gathering the MCP params for our trader

In [3]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [4]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [5]:
tavily_env = {"TAVILY_API_KEY": os.getenv("TAVILY_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "tavily-mcp@latest"], "env": tavily_env}
]

### Now create the MCPServerStdio for each

In [6]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [9]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model=gemini_model,
        mcp_servers=mcp_servers,
    )
    return researcher

In [8]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [12]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here's a summary of the latest news on Amazon:

*   **AWS Layoffs:** Amazon is trimming its Amazon Web Services (AWS) workforce, laying off hundreds of employees due to AI-driven changes and efficiency efforts.
*   **AI Wearables Acquisition:** Amazon is set to acquire Bee, a startup focused on AI-enabled wearable technology, specifically a bracelet designed to transcribe conversations.
*   **Account Security Warning:** A surge in attacks targeting Amazon accounts has been reported, with scammers sending fake text messages regarding orders, deliveries, or refunds. Users are advised to be cautious.
*   **Starfish AI:** Amazon's Starfish AI project aims to gather product data from across the web to improve product listings. This has raised concerns about how Amazon uses product information from other websites.
*   **Fire TV Soundbar Deal:** The Amazon Fire TV Soundbar Plus with a built-in subwoofer is currently available at a discounted price of $180, down from $250.

### Look at the trace

https://platform.openai.com/traces

In [13]:
mindy_initial_strategy = "You are a long term investment trader that focuses on value investing."
Account.get("Mindy").reset(mindy_initial_strategy)

display(Markdown(await read_accounts_resource("Mindy")))
display(Markdown(await read_strategy_resource("Mindy")))

{"name": "mindy", "balance": 10000.0, "strategy": "You are a long term investment trader that focuses on value investing.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-24 16:58:41", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a long term investment trader that focuses on value investing.

### And now - to create our Trader Agent

In [14]:
agent_name = "Mindy"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [15]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Mindy and your account is under your name, Mindy.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a long term investment trader that focuses on value investing.
Your current holdings and balance is:
{"name": "mindy", "balance": 10000.0, "strategy": "You are a long term investment trader that focuses on value investing.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-24 16:58:41", 10000.0], ["2025-07-24 16:59:13", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portf

### And to run our Trader

In [17]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model=gemini_model,
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=10)
display(Markdown(result.final_output))

I bought 2 shares of AAPL for $214.58 each, as a long term investment. My remaining balance is $9570.84.


### Then go and look at the trace

http://platform.openai.com/traces


In [18]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "mindy", "balance": 9570.8434, "strategy": "You are a long term investment trader that focuses on value investing.", "holdings": {"AAPL": 2}, "transactions": [{"symbol": "AAPL", "quantity": 2, "price": 214.5783, "timestamp": "2025-07-24 17:03:38", "rationale": "Long term investment in a stable company."}], "portfolio_value_time_series": [["2025-07-24 16:58:41", 10000.0], ["2025-07-24 16:59:13", 10000.0], ["2025-07-24 17:03:38", 9999.143399999999], ["2025-07-24 17:03:45", 9999.143399999999]], "total_portfolio_value": 9999.143399999999, "total_profit_loss": -0.8566000000009808}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [2]:
from traders import Trader


In [3]:
trader = Trader("Mindy")

In [4]:
trader.model_name

'gemini-2.0-flash'

In [5]:
await trader.run()

In [6]:
await read_accounts_resource("Mindy")

'{"name": "mindy", "balance": 8808.7222, "strategy": "You are a long term investment trader that focuses on value investing.", "holdings": {"AAPL": 2, "GSK": 20}, "transactions": [{"symbol": "AAPL", "quantity": 2, "price": 214.5783, "timestamp": "2025-07-24 17:03:38", "rationale": "Long term investment in a stable company."}, {"symbol": "GSK", "quantity": 20, "price": 38.10606, "timestamp": "2025-07-24 17:15:20", "rationale": "Value investing, undervalued according to Morningstar."}], "portfolio_value_time_series": [["2025-07-24 16:58:41", 10000.0], ["2025-07-24 16:59:13", 10000.0], ["2025-07-24 17:03:38", 9999.143399999999], ["2025-07-24 17:03:45", 9999.143399999999], ["2025-07-24 17:10:31", 9999.143399999999], ["2025-07-24 17:12:59", 9999.143399999999], ["2025-07-24 17:14:51", 9999.143399999999], ["2025-07-24 17:15:20", 9997.6222], ["2025-07-24 17:15:32", 9997.6222]], "total_portfolio_value": 9997.6222, "total_profit_loss": -2.3778000000002066}'

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [7]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("mindy")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 19 tools
